# Завдання 1 — Машинний переклад (Transformer) EN ↔ UK (ManyThings)

Цей ноутбук робить переклад через готові моделі MarianMT + BLEU + приклади + CSV.


In [1]:

import zipfile, random
from pathlib import Path
import pandas as pd
import torch
import sacrebleu

print("Torch:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
device_id = 0 if device == "cuda" else -1
print("Device:", device)


Torch: 2.9.1+cu128
Device: cpu


In [2]:
LAB_DIR = Path(r"/home/kali/Downloads/data_anal/lab4")

zip_path = LAB_DIR / "ukr-eng.zip"
extract_dir = LAB_DIR / "manythings_ukr_eng"
ukr_txt = LAB_DIR / "ukr.txt"

def pick_txt_file() -> Path:
    
    if ukr_txt.exists():
        return ukr_txt

    
    if zip_path.exists():
        extract_dir.mkdir(exist_ok=True)
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(extract_dir)

        txts = sorted(extract_dir.rglob("*.txt"))
        if not txts:
            raise FileNotFoundError("У ukr-eng.zip не знайдено .txt після розпакування.")

        
        for p in txts:
            if p.name.lower() == "ukr.txt":
                return p


        return txts[0]

    
    txts = sorted(LAB_DIR.rglob("*.txt"))
    if not txts:
        raise FileNotFoundError(
            f"Не знайдено ні ukr.txt, ні ukr-eng.zip, ні будь-якого .txt у {LAB_DIR}.\n"
            "Поклади ukr.txt або ukr-eng.zip у цю папку."
        )

    
    for p in txts:
        if "ukr" in p.name.lower():
            return p

    return txts[0]

data_txt = pick_txt_file()
print("Using dataset file:", data_txt)

# Парсимо рядки: очікуємо таби (EN\tUK\t...) або (UK\tEN\t...) ---
def lang_score(s: str):
    latin = sum(ch.isalpha() and ('A' <= ch <= 'Z' or 'a' <= ch <= 'z') for ch in s)
    cyr = sum(ch in "АБВГҐДЕЄЖЗИІЇЙКЛМНОПРСТУФХЦЧШЩЬЮЯабвгґдеєжзиіїйклмнопрстуфхцчшщьюя" for ch in s)
    return latin, cyr

pairs = []
swapped = 0
with open(data_txt, "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        parts = line.rstrip("\n").split("\t")
        if len(parts) < 2:
            continue
        a, b = parts[0].strip(), parts[1].strip()
        if not a or not b:
            continue

        a_lat, a_cyr = lang_score(a)
        b_lat, b_cyr = lang_score(b)

        # якщо (a схоже на EN) і (b схоже на UK) -> (EN, UK)
        if a_lat >= a_cyr and b_cyr >= b_lat:
            en, uk = a, b
        # якщо навпаки -> свап
        elif b_lat >= b_cyr and a_cyr >= a_lat:
            en, uk = b, a
            swapped += 1
        else:
            # неясно — залишаємо як (a,b)
            en, uk = a, b

        pairs.append((en, uk))

print("Total pairs:", len(pairs))
print("Swapped by heuristic:", swapped)
print("Example:", pairs[0])

Using dataset file: /home/kali/Downloads/data_anal/lab4/ukr.txt
Total pairs: 160049
Swapped by heuristic: 0
Example: ('Go.', 'Йди.')


In [3]:

random.seed(42)
random.shuffle(pairs)

MAX_PAIRS = 8000
VAL_N     = 500

pairs = pairs[:MAX_PAIRS]
val_pairs = pairs[:VAL_N]

val_df = pd.DataFrame(val_pairs, columns=["en", "uk"])
val_df.head()


,en,uk
0,It's all so sad.,Це все так сумно.
1,You should've kissed Tom.,Тобі слід було поцілувати Тома.
2,I spent a lot of time listening to music.,"Я провів багато часу, слухаючи музику."
3,I'm sloshed.,Я під мухою.
4,Tom's comfortable.,Тому зручно.


In [5]:

from transformers import pipeline

MODEL_EN_UK = "Helsinki-NLP/opus-mt-en-uk"
MODEL_UK_EN = "Helsinki-NLP/opus-mt-uk-en"

en2uk = pipeline("translation", model=MODEL_EN_UK, device=device_id)
uk2en = pipeline("translation", model=MODEL_UK_EN, device=device_id)

print("Quick examples EN→UK:")
for t in ["Do not open suspicious attachments.", "Please confirm your password using the link."]:
    print("-", t)
    print("  ", en2uk(t, max_length=128)[0]["translation_text"])

print("\nQuick examples UK→EN:")
for t in ["Не відкривайте підозрілі вкладення в листах.", "Будь ласка, підтвердіть пароль за посиланням."]:
    print("-", t)
    print("  ", uk2en(t, max_length=128)[0]["translation_text"])


Device set to use cpu
Device set to use cpu


Quick examples EN→UK:
- Do not open suspicious attachments.
   Не відкривайте підозрілі долучення.
- Please confirm your password using the link.
   Будь ласка, підтвердіть ваш пароль за допомогою посилання.

Quick examples UK→EN:
- Не відкривайте підозрілі вкладення в листах.
   Do not open suspicious letters.
- Будь ласка, підтвердіть пароль за посиланням.
   Please confirm the link password.


In [6]:

# BLEU on validation (EN→UK)
preds, refs = [], []
for en, uk in val_pairs:
    out = en2uk(en, max_length=128)[0]["translation_text"]
    preds.append(out)
    refs.append([uk])

bleu = sacrebleu.corpus_bleu(preds, refs)
print("BLEU:", bleu.score)


BLEU: 100.00000000000004


In [7]:

out_df = val_df.copy()
out_df["uk_pred"] = preds
out_df.to_csv("results_task1_translation_en2uk.csv", index=False, encoding="utf-8")
print("Saved: results_task1_translation_en2uk.csv")
out_df.head()


Saved: results_task1_translation_en2uk.csv


,en,uk,uk_pred
0,It's all so sad.,Це все так сумно.,Це все так сумно.
1,You should've kissed Tom.,Тобі слід було поцілувати Тома.,Тобі слід було поцілувати Тома.
2,I spent a lot of time listening to music.,"Я провів багато часу, слухаючи музику.","Я провів багато часу, слухаючи музику."
3,I'm sloshed.,Я під мухою.,Я приголомшений.
4,Tom's comfortable.,Тому зручно.,Тому зручно.



## mini fine-tune
Якщо треба показати навчання — ставимо `DO_TRAIN=True`.

In [10]:
DO_TRAIN = False

In [11]:

if DO_TRAIN:
    from datasets import Dataset
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
    from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

    train_pairs = pairs[VAL_N:VAL_N+2000]
    train_df = pd.DataFrame(train_pairs, columns=["en", "uk"])

    ds_train = Dataset.from_pandas(train_df)
    ds_val   = Dataset.from_pandas(val_df)

    base_model = "Helsinki-NLP/opus-mt-en-uk"
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    model = AutoModelForSeq2SeqLM.from_pretrained(base_model).to(device)

    max_len = 96

    def preprocess(batch):
        x = tokenizer(batch["en"], truncation=True, max_length=max_len)
        y = tokenizer(text_target=batch["uk"], truncation=True, max_length=max_len)
        x["labels"] = y["input_ids"]
        return x

    tok_train = ds_train.map(preprocess, batched=True, remove_columns=ds_train.column_names)
    tok_val   = ds_val.map(preprocess, batched=True, remove_columns=ds_val.column_names)

    collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

    args = Seq2SeqTrainingArguments(
        output_dir="mt_en_uk_mini_finetune",
        overwrite_output_dir=True,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        learning_rate=5e-5,
        max_steps=30,
        logging_steps=5,
        eval_strategy="no",
        save_strategy="no",
        predict_with_generate=True,
        report_to="none",
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=tok_train,
        eval_dataset=tok_val,
        data_collator=collator,
    )

    trainer.train()
    model.save_pretrained("mt_en_uk_mini_finetune/final")
    tokenizer.save_pretrained("mt_en_uk_mini_finetune/final")
    print("Saved finetuned to mt_en_uk_mini_finetune/final")


/home/kali/Downloads/data_anal/lab4/.venv/lib/python3.13/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/home/kali/Downloads/data_anal/lab4/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
5,0.790700
10,0.582200
15,0.582400
20,0.453500
25,0.602900
30,0.687500


/home/kali/Downloads/data_anal/lab4/.venv/lib/python3.13/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61586]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Saved finetuned to mt_en_uk_mini_finetune/final
